### Installation
    pip install pandas openpyxl

In [1]:
import pandas as pd
from openpyxl.styles import Font

In [2]:
df1 = pd.read_excel("input_file_01.xlsx")
df2 = pd.read_excel("input_file_02.xlsx")

In [3]:
df1.head(5)

,id,product,salesdate,country,salesvolume
0,IDF_1,Carrot,2018-04-29 08:42:39.411,NaN,410411
1,IDF_10,Celery,2018-04-29 11:27:15.277,ABW,232277
2,IDF_11,Celery,2018-04-29 11:17:36.124,ABW,110124
3,IDF_2,Celery,2018-04-29 09:24:18.993,ABW,932993
4,IDF_3,Celery,2018-04-29 09:45:30.373,ABW,310373


In [4]:
df2.head(5)

,country,salesmonth,potato_salesvolume
0,ABW,2018-05-31,2055
1,TTO,2018-05-31,2200
2,BRB,2018-05-31,1201
3,CUW,2018-05-31,502
4,ABW,2018-04-30,2140


In [5]:
merged_df = df1.merge(df2, on="country")

merged_df.head(5)

,id,product,salesdate,country,salesvolume,salesmonth,potato_salesvolume
0,IDF_10,Celery,2018-04-29 11:27:15.277,ABW,232277,2018-05-31,2055
1,IDF_10,Celery,2018-04-29 11:27:15.277,ABW,232277,2018-04-30,2140
2,IDF_11,Celery,2018-04-29 11:17:36.124,ABW,110124,2018-05-31,2055
3,IDF_11,Celery,2018-04-29 11:17:36.124,ABW,110124,2018-04-30,2140
4,IDF_2,Celery,2018-04-29 09:24:18.993,ABW,932993,2018-05-31,2055


In [6]:
grouped_df = (
    merged_df.groupby(["country", "product"])["salesvolume"].sum().reset_index()
)

grouped_df

,country,product,salesvolume
0,ABW,Carrot,9231458
1,ABW,Celery,41228972
2,ABW,Radish,5077686
3,BRB,Celery,3785862
4,BRB,Radish,1708484
5,CUW,Celery,20146
6,TTO,Carrot,1621602
7,TTO,Celery,22213632
8,TTO,Radish,1415486


In [7]:
pivot_df = grouped_df.pivot(index="country", columns="product", values="salesvolume")

pivot_df

product,Carrot,Celery,Radish
country,,,
ABW,9231458.0,41228972.0,5077686.0
BRB,NaN,3785862.0,1708484.0
CUW,NaN,20146.0,NaN
TTO,1621602.0,22213632.0,1415486.0


In [8]:
# To get total sales volume, by country
total_sales_volume = merged_df.groupby("country")["salesvolume"].sum().reset_index()
total_sales_volume = total_sales_volume.rename(
    columns={"salesvolume": "Sales volume (1000s)"}
)
total_sales_volume = total_sales_volume.set_index("country")

In [9]:
writer = pd.ExcelWriter("output_file.xlsx")

pivot_df.to_excel(writer, sheet_name="Monthly Sales", startrow=5, startcol=1)
total_sales_volume.to_excel(writer, sheet_name="Monthly Sales", startrow=14, startcol=1)

In [10]:
workbook = writer.book
worksheet = workbook.active
print(worksheet)

<Worksheet "Monthly Sales">


In [12]:
worksheet["A1"] = "Report name"
worksheet["B1"] = "Monthly Sales"
worksheet["A2"] = "Last updated"

worksheet["A5"] = "Sales volume by product"
worksheet["A13"] = "Total sales volume"

bold_font = Font(bold=True)
for cell in ("A1", "B1", "A2", "A5", "A13"):
    worksheet[cell].font = bold_font

workbook.save("output_file.xlsx")